In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image, ImageDraw
import sys
import time

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "VISON_KEY" #vison key
endpoint = "VISON_ENDPOINT" #vision endpoint

In [ ]:
import requests
from io import BytesIO
from IPython.display import display

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'''
OCR: Read File using the Read API, extract text - remote
This example will extract text in an image, then print results, line by line.
This API call can also extract handwriting style text (not shown).
'''
print("===== Read File - remote =====")
# Get an image with text
read_image_url = "https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png"
#read_image_url = "https://d5q4akjun1yjt.cloudfront.net/assets/ocr_header_image-768x402.png"
#read_image_url = "https://azurelessons.com/wp-content/uploads/2020/10/Azure-Cognitive-Services-OCR.jpg"

# Get the image from the URL
response = requests.get(read_image_url)

# Open the image using PIL
img = Image.open(BytesIO(response.content))

# Display the image
display(img)

In [ ]:
# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

draw = ImageDraw.Draw(img)
# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
            x1, y1, x2, y2, x3, y3, x4, y4 = line.bounding_box
            draw.rectangle([x1, y1, x3, y3], outline="red", width=3)

# Display the image with rectangles
display(img)

print()
'''
END - Read File - remote
'''

print("End of Computer Vision quickstart.")